In [7]:
import pandas as pd
import seaborn as sns
import numpy as np
import json
import sys
import requests
# setting path
sys.path.append('../common')
import utils
import limitUtils
import fillInUtils

SETUP_PATH = 'setup.json'
f = open(SETUP_PATH)
data = json.load(f)

### Fill Dotnet

In [10]:
response = requests.get(data['app']["url"])
content = response.content.decode("utf-8")
len(json.loads(content))

100000

In [9]:
fillInUtils.fill_framework(
    app=data['app'],
    N = data['fill_in_quantity'],
)

http://localhost:5029/Member/clear/
b'Count of Members: 0'
200
http://localhost:5029/Member/fill-in/
100000
iterations: 100
1/100: b'Count of Members: 1000'
1/100: 200
1/100: duration: 1s
1/100: expected end in: 1m 39s
2/100: b'Count of Members: 2000'
2/100: 200
2/100: duration: 0s
2/100: expected end in: 0s
3/100: b'Count of Members: 3000'
3/100: 200
3/100: duration: 0s
3/100: expected end in: 0s
4/100: b'Count of Members: 4000'
4/100: 200
4/100: duration: 0s
4/100: expected end in: 0s
5/100: b'Count of Members: 5000'
5/100: 200
5/100: duration: 0s
5/100: expected end in: 0s
6/100: b'Count of Members: 6000'
6/100: 200
6/100: duration: 0s
6/100: expected end in: 0s
7/100: b'Count of Members: 7000'
7/100: 200
7/100: duration: 0s
7/100: expected end in: 0s
8/100: b'Count of Members: 8000'
8/100: 200
8/100: duration: 0s
8/100: expected end in: 0s
9/100: b'Count of Members: 9000'
9/100: 200
9/100: duration: 0s
9/100: expected end in: 0s
10/100: b'Count of Members: 10000'
10/100: 200
10/100

## Run k6 test script

In [11]:
limitUtils.test_vus_limit(
    result_base_dir=data["test_result_base_dir"],
    test_result_dir_prefix=data["test_result_dir_prefix"],
    script_path=data["script_path"],
    setup_path=SETUP_PATH,
    app=data["app"],
    inital_vus=64,
    out_file_name="out_fill_in.txt",
    end=None
)

date and time = 2024-06-01_18-00-05
./out/out_2024-06-01_18-00-05

          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

     execution: local
        script: ../../grafana-k6/script.js
        output: json (./out/out_2024-06-01_18-00-05/dotnet.json), csv (./out/out_2024-06-01_18-00-05/dotnet.csv)

     scenarios: (100.00%) 1 scenario, 64 max VUs, 1m0s max duration (incl. graceful stop):
              * default: 64 looping VUs for 30s (gracefulStop: 30s)


running (0m01.0s), 64/64 VUs, 0 complete and 0 interrupted iterations
default   [   3% ] 64 VUs  01.0s/30s

running (0m02.0s), 64/64 VUs, 0 complete and 0 interrupted iterations
default   [   7% ] 64 VUs  02.0s/30s

running (0m03.0s), 64/64 VUs, 0 complete and 0 interrupted iterations
default   [  10% ] 64 VUs  03.0s/30s

running (0m04.0s), 64/64 VUs, 0 complete and 0 interrupted iterations
defaul